In [35]:
import cv2
from mtcnn import MTCNN
import numpy as np
import random

# Inicializar el detector MTCNN
mtcnn_det = MTCNN()

# Cargar las imágenes de las marcas de colores y la bufanda
pintura_derecha = cv2.imread('images/pinturacara1.png', cv2.IMREAD_UNCHANGED)
pintura_izquierda = cv2.imread('images/pinturacara2.png', cv2.IMREAD_UNCHANGED)
bufanda = cv2.imread('images/bufanda.png', cv2.IMREAD_UNCHANGED)
escudo = cv2.imread('images/escudo.png', cv2.IMREAD_UNCHANGED)


# Crear lista de partículas de confeti
confeti = []

def generar_confeti():
    # Genera un confeti con una posición aleatoria en la parte superior del frame
    return {'x': random.randint(0, 640), 'y': 0, 
            'velocidad': random.uniform(10, 20),  # Diferentes velocidades más amplias
            'color': random.choice([(255, 0, 0), (0, 255, 255)])}  # Amarillo y azul

# Actualiza las posiciones del confeti y aumenta su velocidad según su caída
def actualizar_confeti():
    for part in confeti:
        # Aumenta la velocidad en función de la posición vertical (y)
        # Cuanto más baja esté la partícula, más rápido caerá
        part['velocidad'] += 1  # Incrementar la velocidad de caída

        # Mueve el confeti hacia abajo
        part['y'] += part['velocidad']
    
    # Elimina el confeti que ya ha salido de la pantalla
    confeti[:] = [part for part in confeti if part['y'] < 480]

# Dibuja las partículas de confeti
def dibujar_confeti(frame):
    for part in confeti:
        # Dibuja cada partícula de confeti como un círculo
        cv2.circle(frame, (int(part['x']), int(part['y'])), 5, part['color'], -1)

def superponer_imagen(bg, fg, x, y, w, h):
    # Redimensionar la imagen de la bufanda al tamaño deseado
    fg = cv2.resize(fg, (w, h))

    # Limitar las coordenadas de la superposición para que no excedan los límites del frame
    bg_h, bg_w = bg.shape[:2]
    x1, x2 = max(0, x), min(x + w, bg_w)
    y1, y2 = max(0, y), min(y + h, bg_h)
    
    # Si la imagen está completamente fuera de los bordes, no hacer nada
    if x1 >= x2 or y1 >= y2:
        return

    # Recortar la imagen si está fuera de los límites
    fg_x1, fg_x2 = 0, x2 - x1
    fg_y1, fg_y2 = 0, y2 - y1
    fg_cropped = fg[fg_y1:fg_y2, fg_x1:fg_x2]

    # Superponer la imagen con transparencia (asumiendo que fg tiene un canal alfa)
    for i in range(3):  # Canales RGB
        bg[y1:y2, x1:x2, i] = fg_cropped[:, :, i] * (fg_cropped[:, :, 3] / 255.0) + \
                              bg[y1:y2, x1:x2, i] * (1 - fg_cropped[:, :, 3] / 255.0)

# Capturar video desde la cámara
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el frame a RGB (MTCNN espera imágenes en RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detectar caras y puntos clave
    results = mtcnn_det.detect_faces(rgb_frame)

    # Verificar si hay resultados
    if results:
        for result in results:
            # Dibujar la caja delimitadora de la cara
            x, y, width, height = result['box']
            #cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 0, 255), 2)

            # Obtener los puntos clave (ojos, nariz, boca)
            keypoints = result.get('keypoints', {})
            if keypoints:
                left_eye = keypoints.get('left_eye')
                right_eye = keypoints.get('right_eye')
                nose = keypoints.get('nose')
                mouth_left = keypoints.get('mouth_left')
                mouth_right = keypoints.get('mouth_right')

                if left_eye and right_eye:
                    # Calcular las posiciones para las marcas de colores debajo de los ojos
                    pintura_izquierda_x = left_eye[0] - 30  # Ajustar horizontalmente
                    pintura_izquierda_y = left_eye[1] + 15  # Colocar debajo del ojo izquierdo

                    pintura_derecha_x = right_eye[0] - 15  # Ajustar horizontalmente
                    pintura_derecha_y = right_eye[1] + 15  # Colocar debajo del ojo derecho

                    # Superponer las marcas debajo de los ojos
                    superponer_imagen(frame, pintura_izquierda, pintura_izquierda_x, pintura_izquierda_y, 50, 25)
                    superponer_imagen(frame, pintura_derecha, pintura_derecha_x, pintura_derecha_y, 50, 25)

                if mouth_left and mouth_right:
                    # Calcular la distancia entre mouth_left y mouth_right
                    mouth_width = np.linalg.norm(np.array(mouth_left) - np.array(mouth_right))

                    # Estimación de la altura de la boca
                    mouth_height = abs(mouth_left[1] - nose[1])  # Usar la nariz como referencia para la boca superior

                    # Calcular el tamaño del escudo basado en el ancho de la cara
                    escudo_width = int(width * 1.75)  # Ajustar el ancho del escudo en función del tamaño de la cara
                    escudo_height = int(escudo_width * escudo.shape[0] / escudo.shape[1])  # Mantener la proporción del escudo

                    # Calcular la posición para superponer el escudo encima de la cabeza
                    escudo_x = x - (escudo_width - width) // 2  # Centrar el escudo en el rostro
                    escudo_y = y - escudo_height  # Colocar el escudo encima de la cabeza

                    # Umbral para determinar si la boca está abierta o cerrada
                    if mouth_height > mouth_width * 0.75:
                        # cv2.putText(frame, "Boca abierta", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        superponer_imagen(frame, escudo, escudo_x, escudo_y, escudo_width, escudo_height)
                        if len(confeti) < 75:  # Limitar el número de partículas
                            confeti.append(generar_confeti())

                    # Actualizar y dibujar el confeti detrás de la cara y bufanda
                    actualizar_confeti()
                    dibujar_confeti(frame)
                    
                    #else:                        
                        #cv2.putText(frame, "Boca cerrada", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    # Superponer la bufanda en el cuello
                    # Posicionar la bufanda debajo de la boca, ajustando en el eje Y
                    bufanda_width = int(width * 1.90)  # Ajustar el ancho de la bufanda al ancho de la cara
                    bufanda_height = int(bufanda_width * bufanda.shape[0] / bufanda.shape[1])  # Mantener la proporción de la bufanda
                    bufanda_x = x - (bufanda_width - width) + width//4  # Posicionar la bufanda horizontalmente centrada en la cara
                    bufanda_y = y + int(height//1.6)  # Colocar la bufanda justo debajo de la cara

                    # Superponer la bufanda en la imagen
                    superponer_imagen(frame, bufanda, bufanda_x, bufanda_y, bufanda_width, bufanda_height)


    # Mostrar el frame con los recuadros, bufanda y marcas de colores
    cv2.imshow("Detección de Rostros con Bufanda y Marcas", frame)

    # Salir si se presiona 'Esc'
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()
